In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import os
import sys
import torch
from tqdm import tqdm
import copy
# import h5py
import numpy as np
# import pandas as pd
from IPython.display import display
import random
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np
# import sklearn


# local imports
sys.path.insert(
    0,
    os.path.join(
        os.path.dirname(os.path.abspath('')), "src"
    )
)
pass
sys.path.pop(0)


'/scratch_local/arubinstein17-163577/starlight/src'

In [12]:
def get_link(raw_link):
    id = raw_link.split("/d/")[-1].split("/")[0]
    return f"https://drive.google.com/uc?id={id}"

In [13]:
get_link("https://drive.google.com/file/d/1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28/view?usp=drive_link")

'https://drive.google.com/uc?id=1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28'

In [8]:
!gdown https://drive.google.com/uc?id=1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28

Downloading...
From (original): https://drive.google.com/uc?id=1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28
From (redirected): https://drive.google.com/uc?id=1g-TxEGbORtHmxVEefoJtk2yxSf_mHL28&confirm=t&uuid=98788ac5-cd93-4eba-bc9b-97a69c1064f5
To: /scratch_local/arubinstein17-163577/starlight/notebooks/cifar10_resnet18.zip
100%|███████████████████████████████████████| 4.37G/4.37G [00:42<00:00, 104MB/s]


In [10]:
# !tar -xzf /scratch_local/arubinstein17-163577/starlight/notebooks/cifar10_resnet18.zip
!unzip cifar10_resnet18.zip -d ./

Archive:  cifar10_resnet18.zip
   creating: ./cifar10_resnet18/
   creating: ./cifar10_resnet18/anchors/
  inflating: ./cifar10_resnet18/anchors/2oo0upw1.pt  
  inflating: ./cifar10_resnet18/anchors/3m2c3guk.pt  
  inflating: ./cifar10_resnet18/anchors/448kfne2.pt  
  inflating: ./cifar10_resnet18/anchors/52l22gc1.pt  
  inflating: ./cifar10_resnet18/anchors/5791j5is.pt  
  inflating: ./cifar10_resnet18/anchors/5hrwt8sd.pt  
  inflating: ./cifar10_resnet18/anchors/6ids7w54.pt  
  inflating: ./cifar10_resnet18/anchors/6kxd228s.pt  
  inflating: ./cifar10_resnet18/anchors/7sckybft.pt  
  inflating: ./cifar10_resnet18/anchors/7y4wdzel.pt  
  inflating: ./cifar10_resnet18/anchors/8wmcs3t5.pt  
  inflating: ./cifar10_resnet18/anchors/ahszhrke.pt  
  inflating: ./cifar10_resnet18/anchors/atn3ky2k.pt  
  inflating: ./cifar10_resnet18/anchors/ce0idfac.pt  
  inflating: ./cifar10_resnet18/anchors/e4fr64ya.pt  
  inflating: ./cifar10_resnet18/anchors/eq157ykm.pt  
  inflating: ./cifar10_resnet18

In [ ]:
def compute_margin(model_1, model_2, class_i, class_j, x, norm="l1"):
    # d = |f_i(x) - f_j(x)| / |grad_x f_i(x) - grad_x f_j (x)|_1
    def process_x(model, class_i, x):
        assert x.requires_grad == True
        with torch.enable_grad():
            output = model(x)[..., class_i]

            # grad = torch.autograd.grad(
            #     output,
            #     x,
            #     create_graph=False
            # )[0]
        output.backward(input=x)
        grad = x.grad
        x.grad = None
        return output, grad

    x.requires_grad = True

    output_1, grad_1 = process_x(model_1, class_i, x)
    output_2, grad_2 = process_x(model_2, class_j, x)

    # assert x.grad = None
    assert norm == "l1"
    if norm == "l1":
        return (
                (output_1 - output_2).abs()
            /
                (grad_1(x) - grad_2(x)).abs().sum()
        )
    else:
        raise NotImplementedError("Only L1 norm is supported")